In [7]:
import pandas as pd
import numpy as np
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
#import get to call a get request on the site
from requests import get
import re

In [52]:
def scrape(url):
        #get the first page of motorcyle listings
        response = get(url)
        html_soup = BeautifulSoup(response.text, 'html.parser')
        #find the total number of posts to find the limit of the pagination
        results_num = html_soup.find('div', class_= 'search-legend')
        results_total = int(results_num.find('span', class_='totalcount').text) #pulled the total count of posts as the upper bound of the pages array

        #each page has 119 posts so each new page is defined as follows: s=120, s=240, s=360, and so on. So we need to step in size 120 in the np.arange function
        pages = np.arange(0, results_total+1, 120)
        links = []

        for page in pages:
            #get request
            links.append(url 
                           + "?s=" #the parameter for defining the page number 
                           + str(page)) #the page number in the pages array from earlier
        price_list= []
        url_list = []
        title_list = []

        for link in links:
            response = get(link)
            html_soup = BeautifulSoup(response.text, 'html.parser')
            results = html_soup.find(class_='rows')
            results.prettify()
            bike_elements = results.find_all('li', class_ ='result-row')
            for bike_elem in bike_elements:

                #   print(bike_elem.text)
                price_elem =bike_elem.find('span', class_='result-price')
                try:
                    url_elem =bike_elem.find('a', class_='result-image gallery')['href']
                except TypeError:
                    pass
                title_elem =bike_elem.find('a', class_="result-title hdrlnk")
                title_list.append(title_elem.text.strip())
                url_list.append(url_elem)
                price_list.append(price_elem.text.strip())

        bike_title_2= []
        attribute_list =[]

        for i, url in enumerate(url_list):    
            bike_url = url
            bike_page = requests.get(bike_url)
            bike_soup =BeautifulSoup(bike_page.content, 'html.parser')
            attributes = bike_soup.find_all('p', class_='attrgroup')
        #     for attribute in attributes:
            if attributes:
                bike_title_2.append(attributes[0].text.strip())
                attribute_list.append((attributes[1].text.strip()))
            else:
                bike_title_2.append('none')
                attribute_list.append('none')
                print(i, url)
                
        df = pd.DataFrame(list(zip(price_list, url_list, title_list, bike_title_2, attribute_list, )),
                          columns=['Price','URL', 'title', 'bikeTitle', 'attributes'])

        return df


In [123]:
url = 'https://Houston.craigslist.org/d/motorcycles-scooters/search/mca'

In [54]:
scrape(url)

261 https://washingtondc.craigslist.org/nva/mcy/d/ashburn-2000-harley-road-king/7383446985.html


,Price,URL,title,bikeTitle,attributes
0,"$5,000",https://washingtondc.craigslist.org/nva/mcy/d/...,Beautiful 2007 Vulcan 900,2007 Vulcan 900,condition: like new\n\nengine displacement (CC...
1,"$4,200",https://washingtondc.craigslist.org/nva/mcy/d/...,1994 Honda Goldwing SE,1994 honda goldwing,condition: good\n\nengine displacement (CC): 1...
2,"$10,450",https://washingtondc.craigslist.org/nva/mcd/d/...,2019 Kawasaki Ninja ZX-6R KRT,kawasaki ninja zx-6r krt,delivery available\n\nfuel: gas\n\nodometer: 6...
3,$60,https://washingtondc.craigslist.org/mld/mcy/d/...,scooter/service/repair,vespa,engine displacement (CC): 50\n\nfuel: gas\n\no...
4,"$19,288",https://washingtondc.craigslist.org/doc/mcd/d/...,2021 Harley-Davidson Road King FLHR,2021 Harley-Davidson Road King FLHR,VIN: 1HD1FBC14MB631412\n\nfuel: gas\n\nodomete...
...,...,...,...,...,...
324,$300,https://washingtondc.craigslist.org/doc/mcy/d/...,2017 Street Glider Harley and Manual motorcycl...,2005 harley davidson 1200 sportster,condition: excellent\n\nengine displacement (C...
325,"$3,650",https://washingtondc.craigslist.org/doc/mcy/d/...,BETA Rev3 270 Trials bike,Beta Rev3 270,condition: good\n\ndelivery available\n\nengin...
326,"$16,500",https://washingtondc.craigslist.org/nva/mcy/d/...,2012 Victory Vegas Jackpot Special Edition,2012 victory vegas jackpot,condition: like new\n\nengine displacement (CC...
327,"$13,988",https://washingtondc.craigslist.org/doc/mcd/d/...,2017 Harley-Davidson Softail Heritage Classic ...,2017 Harley-Davidson Softail,VIN: 1HD1BWV18HB048671\n\nfuel: gas\n\nodomete...


In [62]:
# def clean(df):
#     #splitting attributes and making columns to amke more features
#     for row in range(len(df)):
#         attributes = df['attributes'][row].split(', ')[1:]
#         for attribute in attributes:
#             sub_attributes = attribute.split(': ')
#             try:
#                 df.at[row,sub_attributes[0]]= sub_attributes[1]
#             except:
#                 pass
#     #Cleaning each feature
#     for i in range(len(attribute_list)):
#         df['attributes'][i] = df['attributes'][i].strip()
#         df['attributes'][i] = df['attributes'][i].replace('\n', '')
#         df['attributes'][i] = df['attributes'][i].replace('VIN', ', VIN')
#         df['attributes'][i] = df['attributes'][i].replace('condition', ', condition')
#         df['attributes'][i] = df['attributes'][i].replace('fuel', ', fuel')
#         df['attributes'][i] = df['attributes'][i].replace('paint', ', paint')
#         df['attributes'][i] = df['attributes'][i].replace('title', ', title')
#         df['attributes'][i] = df['attributes'][i].replace('engine', ', engine')
#         df['attributes'][i] = df['attributes'][i].replace('odometer', ', odometer')
#         df['attributes'][i] = df['attributes'][i].replace('delivery', ', delivery')
#         df['attributes'][i] = df['attributes'][i].replace('transmission', ', transmission')
#         df['attributes'][i] = df['attributes'][i].replace('type', ', type')
    
#     #getting year from BikeTitle in case it doesnt have it in the title
#     for row in range(len(df)):
#         df.at[row, 'years'] = df['bikeTitle'][row][:4]
        
#     def regex_year(string):
#         try:
#             return re.match(r"[\d]+", string).group()
#         except:
#             return re.match(r"[\d]+", string)
#     #using the above function to get the year from the title
#     df['year'] = list(map(regex_year, df['title']))
#     df=df
    
# #     return print(df)

In [65]:
df = pd.DataFrame(list(zip(price_list,url_list,title_list,bike_title_2, attribute_list, )),columns=['Price','URL', 'title', 'bikeTitle', 'attributes'])
df

NameError: name 'price_list' is not defined

In [66]:
df

NameError: name 'df' is not defined

In [124]:
#get the first page of motorcyle listings
response = get(url)
html_soup = BeautifulSoup(response.text, 'html.parser')
#find the total number of posts to find the limit of the pagination
results_num = html_soup.find('div', class_= 'search-legend')
results_total = int(results_num.find('span', class_='totalcount').text) #pulled the total count of posts as the upper bound of the pages array

#each page has 119 posts so each new page is defined as follows: s=120, s=240, s=360, and so on. So we need to step in size 120 in the np.arange function
pages = np.arange(0, results_total+1, 120)

In [125]:
#get the links for each pagination
links = []

for page in pages:
    #get request
    links.append(url 
                   + "?s=" #the parameter for defining the page number 
                   + str(page)) #the page number in the pages array from earlier


                         
links

['https://Houston.craigslist.org/d/motorcycles-scooters/search/mca?s=0',
 'https://Houston.craigslist.org/d/motorcycles-scooters/search/mca?s=120',
 'https://Houston.craigslist.org/d/motorcycles-scooters/search/mca?s=240',
 'https://Houston.craigslist.org/d/motorcycles-scooters/search/mca?s=360',
 'https://Houston.craigslist.org/d/motorcycles-scooters/search/mca?s=480']

In [126]:
price_list= []
url_list = []
title_list = []

for link in links:
    response = get(link)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    results = html_soup.find(class_='rows')
    results.prettify()
    bike_elements = results.find_all('li', class_ ='result-row')
    for bike_elem in bike_elements:
        
        #   print(bike_elem.text)
        price_elem =bike_elem.find('span', class_='result-price')
        try:
            url_elem =bike_elem.find('a', class_='result-image gallery')['href']
        except TypeError:
            pass
        title_elem =bike_elem.find('a', class_="result-title hdrlnk")
        title_list.append(title_elem.text.strip())
        url_list.append(url_elem)
        price_list.append(price_elem.text.strip())
    #   print(price_list)
        print(title_elem.text.strip())
        print(url_elem)
        print(price_elem.text.strip())
        print()

1992 Harley Davidson Dyna Lowrider Dayton
https://houston.craigslist.org/mcd/d/alvin-1992-harley-davidson-dyna/7394631875.html
$0

2019 Honda Grom
https://houston.craigslist.org/mcy/d/houston-2019-honda-grom/7394630626.html
$3,700

2016 Harley-Davidson FLHXS - Street Glide Special V Twin 1689.5
https://houston.craigslist.org/mcd/d/south-houston-2016-harley-davidson/7394618370.html
$23,199

2012 Dodge Avenger Sedan
https://houston.craigslist.org/mcd/d/alvin-2012-dodge-avenger-sedan/7394598055.html
$0

2007 Victory Jackpot
https://houston.craigslist.org/mcd/d/alvin-2007-victory-jackpot/7394561894.html
$0

2013 Goldwing F6B
https://houston.craigslist.org/mcy/d/houston-2013-goldwing-f6b/7394556622.html
$10,500

1960 Matchless
https://houston.craigslist.org/mcy/d/willis-1960-matchless/7394549826.html
$6,000

2009 Kawasaki Ninja 250
https://houston.craigslist.org/mcd/d/alvin-2009-kawasaki-ninja-250/7394523279.html
$0

2015 Yamaha YZ250F - Less than 3 hours
https://houston.craigslist.org/mcy/

Harley Davidson Ultra Limited
https://houston.craigslist.org/mcy/d/spring-harley-davidson-ultra-limited/7391426693.html
$22,500

2019 Yamaha MT-09 09 Horizontal In-line 847 cc
https://houston.craigslist.org/mcd/d/south-houston-2019-yamaha-mt-horizontal/7391426188.html
$8,445

2006 Harley Davidson Flstci Heritage Soft
https://houston.craigslist.org/mcd/d/alvin-2006-harley-davidson-flstci/7391407936.html
$0

BMW R1150GS
https://houston.craigslist.org/mcy/d/sugar-land-bmw-r1150gs/7391403866.html
$6,500

1992 Harley Davidson Dyna Lowrider Dayton
https://houston.craigslist.org/mcd/d/alvin-1992-harley-davidson-dyna/7391366848.html
$0

2012 Dodge Avenger Sedan
https://houston.craigslist.org/mcd/d/alvin-2012-dodge-avenger-sedan/7391325426.html
$0

2020 Indian Chieftain Elite 116 - *CLEAN* - 500+ Credit Accepted!
https://houston.craigslist.org/mcd/d/south-houston-2020-indian-chieftain/7391323081.html
$33,799

Harley Davidson Street Glide
https://houston.craigslist.org/mcy/d/katy-harley-davidson

2014 Harley-Davidson VRSCDX - Night Rod Special  -Premium Dealer!
https://houston.craigslist.org/mcd/d/las-vegas-2014-harley-davidson-vrscdx/7387960857.html
$11,995

2008 Suzuki GSXR1000 GSXR 1000 GSXR-1000 GSX R1000 GSX-R1000
https://houston.craigslist.org/mcd/d/dallas-2008-suzuki-gsxr1000-gsxr-1000/7387947392.html
$8,749

2008 Suzuki Hayabusa
https://houston.craigslist.org/mcd/d/dallas-2008-suzuki-hayabusa/7387947384.html
$8,999

2017 Yamaha R3 R 3 R-3
https://houston.craigslist.org/mcd/d/dallas-2017-yamaha-r3-3-3/7387947375.html
$4,899

2009 Suzuki GSXR600 GSXR 600 GSXR-600 GSX R600 GSX-R600
https://houston.craigslist.org/mcd/d/dallas-2009-suzuki-gsxr600-gsxr-600/7387947355.html
$7,199

2009 Honda CBR600RR CBR 600 RR CBR-600-RR
https://houston.craigslist.org/mcd/d/dallas-2009-honda-cbr600rr-cbr-600-rr/7387947333.html
$7,970

2007 Suzuki GSXR600 GSXR 600 GSXR-600 GSX R600 GSX-R600
https://houston.craigslist.org/mcd/d/dallas-2007-suzuki-gsxr600-gsxr-600/7387947322.html
$6,799

2014 Su

2007 Yamaha R1 R 1 R-1 SILVER
https://houston.craigslist.org/mcd/d/dallas-2007-yamaha-r1-1-1-silver/7385602438.html
$8,399

2007 Honda CBR600RR CBR 600 RR CBR-600-RR
https://houston.craigslist.org/mcd/d/dallas-2007-honda-cbr600rr-cbr-600-rr/7385602422.html
$6,299

2015 Suzuki GSXR750L5 GSXR 750 L 5 GSXR-750-L-5 GSX R750L5 GSX-R750L5
https://houston.craigslist.org/mcd/d/dallas-2015-suzuki-gsxr750l5-gsxr-750-5/7385602364.html
$10,699

2008 Suzuki GSXR600 GSXR 600 GSXR-600 GSX R600 GSX-R600
https://houston.craigslist.org/mcd/d/dallas-2008-suzuki-gsxr600-gsxr-600/7385602230.html
$6,199

2016 Kawasaki Ninja 300
https://houston.craigslist.org/mcd/d/dallas-2016-kawasaki-ninja-300/7385602203.html
$4,149

2020 Kawasaki Ninja ZX14R ZX 14 R ZX-14-R ABS SE Ninja ZX 14R ABS SE
https://houston.craigslist.org/mcd/d/dallas-2020-kawasaki-ninja-zx14r-zx-14/7385601966.html
$13,999

2018 Honda CBR600RR CBR 600 RR CBR-600-RR
https://houston.craigslist.org/mcd/d/dallas-2018-honda-cbr600rr-cbr-600-rr/7385601

Brand new BMW R1250 GS Adventure
https://houston.craigslist.org/mcy/d/sugar-land-brand-new-bmw-r1250-gs/7382424932.html
$23,500

2017 Honda Rebel 500  -Finance it with Instant Credit Approval!
https://houston.craigslist.org/mcd/d/las-vegas-2017-honda-rebel-500-finance/7382401485.html
$5,495

89 KX125 KAWASAKI WITH TITLE
https://houston.craigslist.org/mcy/d/kingwood-89-kx125-kawasaki-with-title/7382369387.html
$5,900

Ducati Monster 1100
https://houston.craigslist.org/mcy/d/houston-ducati-monster-1100/7382331093.html
$6,000

HARLEY DAVIDSON - FLH 1200 -  1974
https://houston.craigslist.org/mcy/d/houston-harley-davidson-flh/7382262961.html
$12,000

Gsxr 600
https://houston.craigslist.org/mcy/d/katy-gsxr-600/7382223213.html
$5,300

SOLD @hillthrillmotoplex Yamaha V Star
https://houston.craigslist.org/mcy/d/katy-sold-hillthrillmotoplex-yamaha-star/7382187584.html
$5,200

2020 Yamaha YZF-R6 Motorcycle
https://houston.craigslist.org/mcy/d/deer-park-2020-yamaha-yzf-r6-motorcycle/7382172718.ht

In [127]:
bike_title_2= []
attribute_list =[]

for i, url in enumerate(url_list):    
    bike_url = url
    bike_page = requests.get(bike_url)
    bike_soup =BeautifulSoup(bike_page.content, 'html.parser')
    attributes = bike_soup.find_all('p', class_='attrgroup')
#     for attribute in attributes:
    if attributes:
        bike_title_2.append(attributes[0].text.strip())
        attribute_list.append((attributes[1].text.strip()))
    else:
        bike_title_2.append('none')
        attribute_list.append('none')
        print(i, url)

423 https://houston.craigslist.org/mcy/d/brazoria-buy-bikes/7383680696.html


In [128]:
print(len(bike_title_2))
print(len(attribute_list))
print(len(price_list))
print(len(title_list))
print(len(url_list))

528
528
528
528
528


In [129]:
df = pd.DataFrame(list(zip(price_list[:1999],url_list[:1999],title_list[:1999],bike_title_2, attribute_list, )),columns=['Price','URL', 'title', 'bikeTitle', 'attributes'])
df

,Price,URL,title,bikeTitle,attributes
0,$0,https://houston.craigslist.org/mcd/d/alvin-199...,1992 Harley Davidson Dyna Lowrider Dayton,1992 Harley Davidson Dyna Lowrider Dayton,VIN: 1HD1GAL10NY305466\n\nfuel: gas\n\nodomete...
1,"$3,700",https://houston.craigslist.org/mcy/d/houston-2...,2019 Honda Grom,2019 honda grom,condition: good\n\nengine displacement (CC): 1...
2,"$23,199",https://houston.craigslist.org/mcd/d/south-hou...,2016 Harley-Davidson FLHXS - Street Glide Spec...,2016 Harley-Davidson FLHXS - Street G,VIN: 1HD1KRM14GB685770\n\nfuel: \n\nodometer: ...
3,$0,https://houston.craigslist.org/mcd/d/alvin-201...,2012 Dodge Avenger Sedan,2012 Dodge Avenger,VIN: 1C3CD2AB9CN111244\n\nfuel: other\n\nodome...
4,$0,https://houston.craigslist.org/mcd/d/alvin-200...,2007 Victory Jackpot,2007 Victory Jackpot,VIN: 5VPXB26D373000132\n\nfuel: gas\n\nodomete...
...,...,...,...,...,...
523,"$27,560",https://houston.craigslist.org/mcd/d/south-hou...,2018 Can-Am Spyder RT Limited Chrome LTD In-li...,2018 Can-Am Spyder RT Limited Chrome,VIN: 2BXNBDD25JV001503\n\nfuel: \n\nodometer: ...
524,"$2,950",https://houston.craigslist.org/mcy/d/spring-20...,2004 Honda Shadow,2007 honda shadow,condition: excellent\n\nfuel: gas\n\nodometer:...
525,"$15,000",https://houston.craigslist.org/mcy/d/magnolia-...,"2008 Honda Goldwing 1800 -Only 1,350 miles",2008 honda goldwing gl1800,condition: like new\n\nfuel: gas\n\nodometer: ...
526,"$3,900",https://houston.craigslist.org/mcy/d/spring-20...,2007 Honda CRF450X,2007 honda crf450x,condition: excellent\n\nengine displacement (C...


In [246]:
# for i in range(len(attribute_list)):
#     df['attributes'][i] = df['attributes'][i].strip()
#     df['attributes'][i] = df['attributes'][i].replace('\n', '')
# df

In [247]:
# for i in range(len(attribute_list)):
#     df['attributes'][i] = df['attributes'][i].strip()
#     df['attributes'][i] = df['attributes'][i].replace('\n', '')
#     df['attributes'][i] = df['attributes'][i].replace('VIN', ', VIN')
#     df['attributes'][i] = df['attributes'][i].replace('condition', ', condition')
#     df['attributes'][i] = df['attributes'][i].replace('fuel', ', fuel')
#     df['attributes'][i] = df['attributes'][i].replace('paint', ', paint')
#     df['attributes'][i] = df['attributes'][i].replace('title', ', title')
#     df['attributes'][i] = df['attributes'][i].replace('engine', ', engine')
#     df['attributes'][i] = df['attributes'][i].replace('odometer', ', odometer')
#     df['attributes'][i] = df['attributes'][i].replace('delivery', ', delivery')
#     df['attributes'][i] = df['attributes'][i].replace('transmission', ', transmission')
#     df['attributes'][i] = df['attributes'][i].replace('type', ', type')
#     df['attributes'][i] = df['attributes'][i].replace('street', ', street')
# df

In [248]:
# df = pd.concat([df, df['attributes'].str.split(', ', expand=True)], axis=1)
# df

In [249]:
# df.insert(loc=len(df.columns), column ='VIN', value =0)
# df.insert(loc=len(df.columns), column ='fuel', value =0)
# df.insert(loc=len(df.columns), column ='condition', value =0)
# df.insert(loc=len(df.columns), column ='odometer', value =0)
# df.insert(loc=len(df.columns), column ='paint color', value =0)
# df.insert(loc=len(df.columns), column ='transmission', value =0)
# df.insert(loc=len(df.columns), column ='title status', value =0)
# df.insert(loc=len(df.columns), column ='type', value =0)
# df.insert(loc=len(df.columns), column ='engine displacement', value =0)
# df.insert(loc=len(df.columns), column ='street legal', value =0)


In [130]:
# def Convert(a):
#     it = iter(a)
#     res_dct = dict(zip(it, it))
#     return res_dct

#Splitting attributes to make it readable
for row in range(len(df)):
    attributes = df['attributes'][row].split(', ')[1:]
    for attribute in attributes:
        sub_attributes = attribute.split(': ')
        try:
            df.at[row,sub_attributes[0]]= sub_attributes[1]
        except:
            pass
        
        

In [131]:
pd.set_option('display.max_columns',None)

In [132]:
df

,Price,URL,title,bikeTitle,attributes
0,$0,https://houston.craigslist.org/mcd/d/alvin-199...,1992 Harley Davidson Dyna Lowrider Dayton,1992 Harley Davidson Dyna Lowrider Dayton,VIN: 1HD1GAL10NY305466\n\nfuel: gas\n\nodomete...
1,"$3,700",https://houston.craigslist.org/mcy/d/houston-2...,2019 Honda Grom,2019 honda grom,condition: good\n\nengine displacement (CC): 1...
2,"$23,199",https://houston.craigslist.org/mcd/d/south-hou...,2016 Harley-Davidson FLHXS - Street Glide Spec...,2016 Harley-Davidson FLHXS - Street G,VIN: 1HD1KRM14GB685770\n\nfuel: \n\nodometer: ...
3,$0,https://houston.craigslist.org/mcd/d/alvin-201...,2012 Dodge Avenger Sedan,2012 Dodge Avenger,VIN: 1C3CD2AB9CN111244\n\nfuel: other\n\nodome...
4,$0,https://houston.craigslist.org/mcd/d/alvin-200...,2007 Victory Jackpot,2007 Victory Jackpot,VIN: 5VPXB26D373000132\n\nfuel: gas\n\nodomete...
...,...,...,...,...,...
523,"$27,560",https://houston.craigslist.org/mcd/d/south-hou...,2018 Can-Am Spyder RT Limited Chrome LTD In-li...,2018 Can-Am Spyder RT Limited Chrome,VIN: 2BXNBDD25JV001503\n\nfuel: \n\nodometer: ...
524,"$2,950",https://houston.craigslist.org/mcy/d/spring-20...,2004 Honda Shadow,2007 honda shadow,condition: excellent\n\nfuel: gas\n\nodometer:...
525,"$15,000",https://houston.craigslist.org/mcy/d/magnolia-...,"2008 Honda Goldwing 1800 -Only 1,350 miles",2008 honda goldwing gl1800,condition: like new\n\nfuel: gas\n\nodometer: ...
526,"$3,900",https://houston.craigslist.org/mcy/d/spring-20...,2007 Honda CRF450X,2007 honda crf450x,condition: excellent\n\nengine displacement (C...


In [133]:
# #getting year from BikeTitle in case it doesnt have it in the title
# for row in range(len(df)):
#     df.at[row, 'year'] = df['bikeTitle'][row][:4]

In [134]:
df

,Price,URL,title,bikeTitle,attributes
0,$0,https://houston.craigslist.org/mcd/d/alvin-199...,1992 Harley Davidson Dyna Lowrider Dayton,1992 Harley Davidson Dyna Lowrider Dayton,VIN: 1HD1GAL10NY305466\n\nfuel: gas\n\nodomete...
1,"$3,700",https://houston.craigslist.org/mcy/d/houston-2...,2019 Honda Grom,2019 honda grom,condition: good\n\nengine displacement (CC): 1...
2,"$23,199",https://houston.craigslist.org/mcd/d/south-hou...,2016 Harley-Davidson FLHXS - Street Glide Spec...,2016 Harley-Davidson FLHXS - Street G,VIN: 1HD1KRM14GB685770\n\nfuel: \n\nodometer: ...
3,$0,https://houston.craigslist.org/mcd/d/alvin-201...,2012 Dodge Avenger Sedan,2012 Dodge Avenger,VIN: 1C3CD2AB9CN111244\n\nfuel: other\n\nodome...
4,$0,https://houston.craigslist.org/mcd/d/alvin-200...,2007 Victory Jackpot,2007 Victory Jackpot,VIN: 5VPXB26D373000132\n\nfuel: gas\n\nodomete...
...,...,...,...,...,...
523,"$27,560",https://houston.craigslist.org/mcd/d/south-hou...,2018 Can-Am Spyder RT Limited Chrome LTD In-li...,2018 Can-Am Spyder RT Limited Chrome,VIN: 2BXNBDD25JV001503\n\nfuel: \n\nodometer: ...
524,"$2,950",https://houston.craigslist.org/mcy/d/spring-20...,2004 Honda Shadow,2007 honda shadow,condition: excellent\n\nfuel: gas\n\nodometer:...
525,"$15,000",https://houston.craigslist.org/mcy/d/magnolia-...,"2008 Honda Goldwing 1800 -Only 1,350 miles",2008 honda goldwing gl1800,condition: like new\n\nfuel: gas\n\nodometer: ...
526,"$3,900",https://houston.craigslist.org/mcy/d/spring-20...,2007 Honda CRF450X,2007 honda crf450x,condition: excellent\n\nengine displacement (C...


In [109]:
# for row in range(len(df)):
#     df.at[row, 'make'] = df['bikeTitle'][row][:4]

In [110]:
# def regex_year(string):
#     try:
#         return re.match(r"[\d]+", string).group()
#     except:
#         return re.match(r"[\d]+", string)

In [111]:
# #getting the year from title
# df['years'] = list(map(regex_year, df['title']))
# df.tail()


In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 914 entries, 0 to 913
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Price       914 non-null    object
 1   URL         914 non-null    object
 2   title       914 non-null    object
 3   bikeTitle   914 non-null    object
 4   attributes  914 non-null    object
dtypes: object(5)
memory usage: 35.8+ KB


In [113]:
# for row in range(len(df)):
#     if df['title']contains
#     df.at[row, 'make'] = df['title'][row]

In [135]:
for i in range(len(attribute_list)):
    df['attributes'][i] = df['attributes'][i].strip()
    df['attributes'][i] = df['attributes'][i].replace('\n', '')
    df['attributes'][i] = df['attributes'][i].replace('VIN', ', VIN')
    df['attributes'][i] = df['attributes'][i].replace('condition', ', condition')
    df['attributes'][i] = df['attributes'][i].replace('fuel', ', fuel')
    df['attributes'][i] = df['attributes'][i].replace('paint', ', paint')
    df['attributes'][i] = df['attributes'][i].replace('title', ', title')
    df['attributes'][i] = df['attributes'][i].replace('engine', ', engine')
    df['attributes'][i] = df['attributes'][i].replace('odometer', ', odometer')
    df['attributes'][i] = df['attributes'][i].replace('delivery', ', delivery')
    df['attributes'][i] = df['attributes'][i].replace('transmission', ', transmission')
    df['attributes'][i] = df['attributes'][i].replace('type', ', type')

In [136]:
df

,Price,URL,title,bikeTitle,attributes
0,$0,https://houston.craigslist.org/mcd/d/alvin-199...,1992 Harley Davidson Dyna Lowrider Dayton,1992 Harley Davidson Dyna Lowrider Dayton,", VIN: 1HD1GAL10NY305466, fuel: gas, odometer:..."
1,"$3,700",https://houston.craigslist.org/mcy/d/houston-2...,2019 Honda Grom,2019 honda grom,", condition: good, engine displacement (CC): 1..."
2,"$23,199",https://houston.craigslist.org/mcd/d/south-hou...,2016 Harley-Davidson FLHXS - Street Glide Spec...,2016 Harley-Davidson FLHXS - Street G,", VIN: 1HD1KRM14GB685770, fuel: , odometer: 0,..."
3,$0,https://houston.craigslist.org/mcd/d/alvin-201...,2012 Dodge Avenger Sedan,2012 Dodge Avenger,", VIN: 1C3CD2AB9CN111244, fuel: other, odomete..."
4,$0,https://houston.craigslist.org/mcd/d/alvin-200...,2007 Victory Jackpot,2007 Victory Jackpot,", VIN: 5VPXB26D373000132, fuel: gas, odometer:..."
...,...,...,...,...,...
523,"$27,560",https://houston.craigslist.org/mcd/d/south-hou...,2018 Can-Am Spyder RT Limited Chrome LTD In-li...,2018 Can-Am Spyder RT Limited Chrome,", VIN: 2BXNBDD25JV001503, fuel: , odometer: 86..."
524,"$2,950",https://houston.craigslist.org/mcy/d/spring-20...,2004 Honda Shadow,2007 honda shadow,", condition: excellent, fuel: gas, odometer: 2..."
525,"$15,000",https://houston.craigslist.org/mcy/d/magnolia-...,"2008 Honda Goldwing 1800 -Only 1,350 miles",2008 honda goldwing gl1800,", condition: like new, fuel: gas, odometer: 13..."
526,"$3,900",https://houston.craigslist.org/mcy/d/spring-20...,2007 Honda CRF450X,2007 honda crf450x,", condition: excellent, engine displacement (C..."


In [137]:
for row in range(len(df)):
    attributes = df['attributes'][row].split(', ')[1:]
    for attribute in attributes:
        sub_attributes = attribute.split(': ')
        try:
            df.at[row,sub_attributes[0]]= sub_attributes[1]
        except:
            pass
        

In [138]:
df

,Price,URL,title,bikeTitle,attributes,VIN,fuel,odometer,paint color,title status,transmission,condition,engine displacement (CC),type
0,$0,https://houston.craigslist.org/mcd/d/alvin-199...,1992 Harley Davidson Dyna Lowrider Dayton,1992 Harley Davidson Dyna Lowrider Dayton,", VIN: 1HD1GAL10NY305466, fuel: gas, odometer:...",1HD1GAL10NY305466,gas,4706,blue,clean,automatic,NaN,NaN,NaN
1,"$3,700",https://houston.craigslist.org/mcy/d/houston-2...,2019 Honda Grom,2019 honda grom,", condition: good, engine displacement (CC): 1...",NaN,gas,2100,bluestreet legal,clean,manual,good,125,standard
2,"$23,199",https://houston.craigslist.org/mcd/d/south-hou...,2016 Harley-Davidson FLHXS - Street Glide Spec...,2016 Harley-Davidson FLHXS - Street G,", VIN: 1HD1KRM14GB685770, fuel: , odometer: 0,...",1HD1KRM14GB685770,,0,,clean,manual,NaN,NaN,NaN
3,$0,https://houston.craigslist.org/mcd/d/alvin-201...,2012 Dodge Avenger Sedan,2012 Dodge Avenger,", VIN: 1C3CD2AB9CN111244, fuel: other, odomete...",1C3CD2AB9CN111244,other,135166,custom,clean,automatic,NaN,NaN,NaN
4,$0,https://houston.craigslist.org/mcd/d/alvin-200...,2007 Victory Jackpot,2007 Victory Jackpot,", VIN: 5VPXB26D373000132, fuel: gas, odometer:...",5VPXB26D373000132,gas,18456,NaN,clean,automatic,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,"$27,560",https://houston.craigslist.org/mcd/d/south-hou...,2018 Can-Am Spyder RT Limited Chrome LTD In-li...,2018 Can-Am Spyder RT Limited Chrome,", VIN: 2BXNBDD25JV001503, fuel: , odometer: 86...",2BXNBDD25JV001503,,867,custom,clean,manual,NaN,NaN,NaN
524,"$2,950",https://houston.craigslist.org/mcy/d/spring-20...,2004 Honda Shadow,2007 honda shadow,", condition: excellent, fuel: gas, odometer: 2...",NaN,gas,25589,blackstreet legal,clean,manual,excellent,NaN,cruiser
525,"$15,000",https://houston.craigslist.org/mcy/d/magnolia-...,"2008 Honda Goldwing 1800 -Only 1,350 miles",2008 honda goldwing gl1800,", condition: like new, fuel: gas, odometer: 13...",NaN,gas,1350,black,clean,manual,like new,NaN,cruiser
526,"$3,900",https://houston.craigslist.org/mcy/d/spring-20...,2007 Honda CRF450X,2007 honda crf450x,", condition: excellent, engine displacement (C...",NaN,gas,1,white,NaN,manual,excellent,450,dirtbike


In [139]:
#getting year from BikeTitle in case it doesnt have it in the title
for row in range(len(df)):
    df.at[row, 'years'] = df['bikeTitle'][row][:4]

In [140]:
import re 

def regex_year(string):
    try:
        return re.match(r"[\d]+", string).group()
    except:
        return re.match(r"[\d]+", string)

In [141]:
df['year'] = list(map(regex_year, df['title']))
df.tail()

,Price,URL,title,bikeTitle,attributes,VIN,fuel,odometer,paint color,title status,transmission,condition,engine displacement (CC),type,years,year
523,"$27,560",https://houston.craigslist.org/mcd/d/south-hou...,2018 Can-Am Spyder RT Limited Chrome LTD In-li...,2018 Can-Am Spyder RT Limited Chrome,", VIN: 2BXNBDD25JV001503, fuel: , odometer: 86...",2BXNBDD25JV001503,,867,custom,clean,manual,NaN,NaN,NaN,2018,2018
524,"$2,950",https://houston.craigslist.org/mcy/d/spring-20...,2004 Honda Shadow,2007 honda shadow,", condition: excellent, fuel: gas, odometer: 2...",NaN,gas,25589,blackstreet legal,clean,manual,excellent,NaN,cruiser,2007,2004
525,"$15,000",https://houston.craigslist.org/mcy/d/magnolia-...,"2008 Honda Goldwing 1800 -Only 1,350 miles",2008 honda goldwing gl1800,", condition: like new, fuel: gas, odometer: 13...",NaN,gas,1350,black,clean,manual,like new,NaN,cruiser,2008,2008
526,"$3,900",https://houston.craigslist.org/mcy/d/spring-20...,2007 Honda CRF450X,2007 honda crf450x,", condition: excellent, engine displacement (C...",NaN,gas,1,white,NaN,manual,excellent,450,dirtbike,2007,2007
527,"$6,500",https://houston.craigslist.org/mcy/d/richmond-...,2010 HONDA STATELINE,2010 honda stateline,", fuel: gas, odometer: 428, paint color: black...",NaN,gas,428,black,clean,manual,NaN,NaN,NaN,2010,2010


In [94]:
pwd

'/Users/avijames/Dropbox/Mac (2)/Documents/Course/capstone/motorcycle_price_prediction'

In [142]:
df.to_pickle('houston_motorcycle')

In [143]:
df_pickle=pd.read_pickle('houston_motorcycle')
df_pickle

,Price,URL,title,bikeTitle,attributes,VIN,fuel,odometer,paint color,title status,transmission,condition,engine displacement (CC),type,years,year
0,$0,https://houston.craigslist.org/mcd/d/alvin-199...,1992 Harley Davidson Dyna Lowrider Dayton,1992 Harley Davidson Dyna Lowrider Dayton,", VIN: 1HD1GAL10NY305466, fuel: gas, odometer:...",1HD1GAL10NY305466,gas,4706,blue,clean,automatic,NaN,NaN,NaN,1992,1992
1,"$3,700",https://houston.craigslist.org/mcy/d/houston-2...,2019 Honda Grom,2019 honda grom,", condition: good, engine displacement (CC): 1...",NaN,gas,2100,bluestreet legal,clean,manual,good,125,standard,2019,2019
2,"$23,199",https://houston.craigslist.org/mcd/d/south-hou...,2016 Harley-Davidson FLHXS - Street Glide Spec...,2016 Harley-Davidson FLHXS - Street G,", VIN: 1HD1KRM14GB685770, fuel: , odometer: 0,...",1HD1KRM14GB685770,,0,,clean,manual,NaN,NaN,NaN,2016,2016
3,$0,https://houston.craigslist.org/mcd/d/alvin-201...,2012 Dodge Avenger Sedan,2012 Dodge Avenger,", VIN: 1C3CD2AB9CN111244, fuel: other, odomete...",1C3CD2AB9CN111244,other,135166,custom,clean,automatic,NaN,NaN,NaN,2012,2012
4,$0,https://houston.craigslist.org/mcd/d/alvin-200...,2007 Victory Jackpot,2007 Victory Jackpot,", VIN: 5VPXB26D373000132, fuel: gas, odometer:...",5VPXB26D373000132,gas,18456,NaN,clean,automatic,NaN,NaN,NaN,2007,2007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,"$27,560",https://houston.craigslist.org/mcd/d/south-hou...,2018 Can-Am Spyder RT Limited Chrome LTD In-li...,2018 Can-Am Spyder RT Limited Chrome,", VIN: 2BXNBDD25JV001503, fuel: , odometer: 86...",2BXNBDD25JV001503,,867,custom,clean,manual,NaN,NaN,NaN,2018,2018
524,"$2,950",https://houston.craigslist.org/mcy/d/spring-20...,2004 Honda Shadow,2007 honda shadow,", condition: excellent, fuel: gas, odometer: 2...",NaN,gas,25589,blackstreet legal,clean,manual,excellent,NaN,cruiser,2007,2004
525,"$15,000",https://houston.craigslist.org/mcy/d/magnolia-...,"2008 Honda Goldwing 1800 -Only 1,350 miles",2008 honda goldwing gl1800,", condition: like new, fuel: gas, odometer: 13...",NaN,gas,1350,black,clean,manual,like new,NaN,cruiser,2008,2008
526,"$3,900",https://houston.craigslist.org/mcy/d/spring-20...,2007 Honda CRF450X,2007 honda crf450x,", condition: excellent, engine displacement (C...",NaN,gas,1,white,NaN,manual,excellent,450,dirtbike,2007,2007
